In [4]:
import time
import tensorflow as tf
import numpy as np
import pandas as pd
from data.data_read import data_read as dr
from params.params import params
from params.config_gpu import config_gpu
import matplotlib.pylab as plt
from pinn.pinn import PINN

ModuleNotFoundError: No module named 'params'

In [ ]:
train_x = dr.train_s_x
train_y = dr.train_s_y

valid_x = dr.valid_s_x
valid_y = dr.valid_s_y

test_x = dr.test_s_x
test_y = dr.test_s_y


ModuleNotFoundError: No module named 'params.params'

In [ ]:
config_gpu(gpu_flg = 1)

# params
f_in, f_out, width, depth, \
w_init, b_init, act, \
lr, opt, \
f_scl, laaf, c, \
w_ini, w_bnd, w_pde, BC, \
f_mntr, r_seed, \
n_epch, n_btch, c_tol = params()

In [ ]:
t_ini = train_x[:,:1]
x_ini = train_x[:,1:2]
n_ini = train_x[:,2:3]
q_ini = train_x[:,3:]
u_ini = train_y

In [ ]:
##Training model
pinn = PINN(t_ini, x_ini, n_ini, q_ini, u_ini,
            f_in, f_out, width, depth, 
            w_init, b_init, act, 
            lr, opt, 
            f_scl, laaf, c, 
            w_ini, w_bnd, w_pde, BC, 
            f_mntr, r_seed)

In [ ]:
t0 = time.time()
with tf.device("/device:GPU:0"):
    pinn.train(epoch = n_epch, batch = n_btch, tol = c_tol)
t1 = time.time()
elps = t1 - t0
print(">>>>> elapse time for training (sec):", elps)
print(">>>>> elapse time for training (min):", elps / 60.)

In [ ]:
### Verify whether the model is reasonable
valid_x = tf.convert_to_tensor(valid_x, dtype = tf.float32)
valid_result = pinn.forward_pass(valid_x).numpy()

In [ ]:
##Test the performace of the model
test_x = tf.convert_to_tensor(test_x, dtype = tf.float32)
test_result = pinn.forward_pass(test_x).numpy()

In [ ]:
t = list(i for i in range(16))
# Plot the Prediction result
plt.figure(figsize=(8, 6))
plt.scatter(t, test_y[:,1], label='Experimental Data', color='blue')
plt.plot(t, test_result[:,1], label='PINN Model', color='red')

plt.xlabel('Time')
plt.ylabel('Biomass Concentration')
plt.legend()
plt.show()